In [ ]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
# WWF ANIMAL DATA

# read in all sheets of ods excel file for animal data
df = pd.read_excel('/home/muskrat/Documents/eco_data_copy/main_eco_data/animal_copy.ods',
                   index_col=0, sheet_name=None, engine=('odf'))


In [ ]:

 
# WWF ANIMAL DATA

# turn each sheet into dataframe
ecoregions = df['ecoregions']
ecoregions.reset_index(inplace=True)
classes = df['class']
common_names = df['common_names']
common_names_bup = df['common_names_bup']
eco_species = df['ecoregion_species']
family = df['family']
genus = df['genus']
order = df['order_']
species = df['species']
# add unique id to each ecoregion and convert from int to str
ranger = [*range(1, 1 + len(ecoregions))]
ecoregion_ids = [str(x) for x in ranger]

ecoregions.insert(0, 'unique_id', ecoregion_ids)

# print(species.head())
# print(genus.head())

# df1 = common_names_bup.merge(common_names, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
# df1


In [ ]:

 
# WWF ANIMAL DATA

# join species and common name dataframes on species id
species_common = pd.merge(species, common_names_bup, on='SPECIES_ID', how='left').reindex(
    columns=['SPECIES_ID', 'SPECIES', 'COMMON_NAME', 'GENUS_ID'])
# find missing value index
print(species_common.loc[pd.isna(species_common["SPECIES"]), :].index)

# print(species_common.head(10))

genus_species = pd.merge(species_common, genus, on='GENUS_ID', how='left').reindex(
    columns=['SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'GENUS', 'SPECIES', 'COMMON_NAME'])
print(genus_species.loc[pd.isna(genus_species["GENUS"]), :].index)

family_genus = pd.merge(genus_species, family, on='FAMILY_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'FAMILY', 'GENUS', 'SPECIES', 'COMMON_NAME'])
print(family_genus.loc[pd.isna(family_genus["FAMILY"]), :].index)

order_family = pd.merge(family_genus, order, on='ORDER_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'COMMON_NAME'])
print(order_family.loc[pd.isna(order_family["ORDER_DESC"]), :].index)

class_order = pd.merge(order_family, classes, on='CLASS_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'COMMON_NAME'])
print(class_order.loc[pd.isna(class_order["CLASS"]), :].index)



In [ ]:

# WWF ANIMAL DATA

# add genus and species columns into one scientific name column
class_order['Scientific_Name'] = class_order['GENUS'].str.cat(
    class_order['SPECIES'], sep=" ")
# print(class_order.loc[pd.isna(class_order["Scientific_Name"]), :].index)

# find all missing values
# missing = class_order[class_order.isna().any(axis=1)]

# fill missing values with unknown
# class_order.fillna('Unknown', inplace=True)

eco_species_merged = pd.merge(class_order, eco_species, on='SPECIES_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_CODE'])

eco_species_miss = eco_species_merged.loc[pd.isna(eco_species_merged["ECOREGION_CODE"]), :].index


ecoregions_merged = pd.merge(eco_species_merged, ecoregions, on='ECOREGION_CODE', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_CODE', 'ECOREGION_NAME', 'unique_id'])

ecoregions_merged.dropna(subset=['unique_id'], inplace=True)
 
# print(ecos.loc[pd.isna(ecos["ECOREGION_NAME"]), :].index)

# ecos.fillna('Unknown', inplace=True)

# create a list of all ecoregion names that apply to same species id
ecoregion_names_species_mapping = ecoregions_merged.groupby('SPECIES_ID')['ECOREGION_NAME'].apply(list).reset_index()


ecoregion_codes_species_mapping = ecoregions_merged.groupby('SPECIES_ID')['ECOREGION_CODE'].apply(list).reset_index()
# print(ecoregion_names_species_mapping.head(15))

ecoregion_id_species_mapping = ecoregions_merged.groupby('SPECIES_ID')['unique_id'].apply(list).reset_index()

final_merge_1 = pd.merge(class_order, ecoregion_names_species_mapping, on='SPECIES_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_NAME'])

final_merge_2 = pd.merge(final_merge_1, ecoregion_codes_species_mapping, on='SPECIES_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_CODE', 'ECOREGION_NAME'])

final_merge_3 = pd.merge(final_merge_2, ecoregion_id_species_mapping, on='SPECIES_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_CODE', 'unique_id', 'ECOREGION_NAME'])

final = final_merge_3[['SPECIES_ID', 'ORDER_DESC', 'FAMILY', 'CLASS', 'Scientific_Name',
               'COMMON_NAME', 'ECOREGION_CODE', 'unique_id', 'ECOREGION_NAME']]


In [ ]:

ecomap = gpd.read_file(
    '/media/muskrat/060A9B8E0A9B78FF/eco_data/wwf_terr_map_data/terr_map.geojson')

 


In [ ]:

marinemap = gpd.read_file(
    '/media/muskrat/060A9B8E0A9B78FF/eco_data/marine_map_data/marine_map.geojson')
# marinemap = gpd.read_file(
#     '/media/muskrat/060A9B8E0A9B78FF/map_final/eco_map.geojson')


In [ ]:

 


ranger = [*range(826, 826+232)]
ranger2 = [*range(1058, 1058+37)]
ranger3 = ranger2 + ranger
marine_ids = [str(x) for x in ranger3]

marinemap.insert(0, 'unique_id', marine_ids)


In [ ]:


 

marinemap.to_file("marine_map.geojson", driver='GeoJSON')


In [ ]:

 


ecomap['name'] = ecomap['ECO_NAME']

ecomap["unique_id"] = ecomap["unique_id"].astype("float32")

ecomap.drop(ecomap[ecomap['unique_id'].isna()].index, inplace=True)

ecomap["unique_id"] = ecomap["unique_id"].astype("int32")

ecomap['unique_id'] = ecomap['unique_id'].apply(str)

ecomap.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

marinemap['name'] = ''

marinemap.loc[marinemap['TYPE'] == 'PPOW', 'name'] = marinemap['PROVINC']

marinemap.loc[marinemap['TYPE'] == 'MEOW', 'name'] = marinemap['ECOREGION']

econames = pd.concat([ecomap[['unique_id', 'name']], marinemap[[
                     'unique_id', 'name']]], axis=0, keys=['unique_id', 'name'])


# econames.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

econames.reset_index(drop=True, inplace=True)


In [ ]:

 
ecokeys = econames.to_json(orient='records', force_ascii=False)
repr(ecokeys)


In [ ]:

 

file = open("ecoregion_keys.json", "w")
file.write(ecokeys)
file.close

In [ ]:



# create dataframe of all values in class column that are mammals
mammals = final.loc[final['CLASS'].isin(['Mammalia'])].reset_index(drop=True)

amphibians = final.loc[final['CLASS'].isin(
    ['Amphibia'])].reset_index(drop=True)

reptiles = final.loc[final['CLASS'].isin(['Reptilia'])].reset_index(drop=True)

birds = final.loc[final['CLASS'].isin(['Aves'])].reset_index(drop=True)

# create dataframe of all missing values
unknown = final.loc[final['CLASS'].isna()]

# create dataframe of all missing common names
common_miss = final.loc[final['COMMON_NAME'].isna()]

final.drop(final.index[[27729, 27779, 27971]], inplace=True,)
final.reset_index(drop=True, inplace=True)


In [ ]:


# convert to json
mammal_json = mammals.to_json(orient='records', force_ascii=False)
repr(mammal_json)

amphibian_json = amphibians.to_json(orient='records', force_ascii=False)
repr(amphibian_json)

reptile_json = reptiles.to_json(orient='records', force_ascii=False)
repr(reptile_json)

bird_json = birds.to_json(orient='records', force_ascii=False)
repr(bird_json)

final_json = final.to_json(orient='records', force_ascii=False)
repr(final_json)

final_miss_json = common_miss.to_json(orient='records', force_ascii=False)
repr(final_miss_json)


In [ ]:

# WWF ANIMAL DATA

# write to files
# file = open("wwf_mammal.json", "w")
# file.write(mammalj)
# file.close

# file = open("wwf_amphibian.json", "w")
# file.write(amphibianj)
# file.close

# file = open("wwf_reptile.json", "w")
# file.write(reptilej)
# file.close

# file = open("birds_string.json", "w")
# file.write(birdj)
# file.close

file = open("wwf_animal_full.json", "w")
file.write(final_json)
file.close

file = open("wwf_animal_full_miss.json", "w")
file.write(final_miss_json)
file.close
